In [67]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

In [56]:
# URL에서 html가져오는 함수
def HTML가져오기(url):
   _html = ""
   resp = requests.get(url) # URL에 GET 요청을 보내서 돌아온 응답(response)
   if resp.status_code == 200: # 200이면 정상적으로 응답이 왔다는 뜻
      _html = resp.text # 거기서 텍스트 즉 html만 가져오고
   return _html # 그걸 반환함

In [57]:
#리뷰 가져오는 함수
def 리뷰_가져오기(URL):
    html = HTML가져오기(URL) # 위에서 정의된 함수
    soup = BeautifulSoup(html, 'html.parser') # beatifuleSoup 라이브러리에 html넣어서 파싱함 
    
    네이버140자평 = soup.find('div', {'class': 'score_result'}) # score_result 클래스를 가진 div 태그
    별점들 = 네이버140자평.find_all("div",{"class": "star_score"}) 
    리뷰들 = 네이버140자평.find_all("div",{"class": "score_reple"})
    따봉들 = 네이버140자평.find_all("div",{"class": "btn_area"})

    
    별점 = [int(별점.em.text) for 별점 in 별점들] # [number + 10 for number in [1,2,3]]의 결과는 [11, 12, 13]
    리뷰 = [리뷰.p.text.replace('BEST', '') for 리뷰 in 리뷰들]
    결과 = list(zip(별점,리뷰)) # list(zip([1,2,3], [4,5,6]))의 결과는 [(1,4), (2,5), (3,6)]
    
    return 결과

In [58]:

영화코드 = '136900'
URL = 'http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code='+ 영화코드 +'&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page='
가져올_페이지_수 = 5

네이버리뷰 = []
for 페이지 in range(1,가져올_페이지_수+1):
    네이버리뷰 += 리뷰_가져오기(URL + str(페이지))
    time.sleep(1.3)
    
네이버평점=[네이버리뷰[i][0] for i in range(len(네이버리뷰))]
네이버감상평=[네이버리뷰[i][1].replace("\n","").replace("\t","").replace("\r","") for i in range(len(네이버리뷰))]    
# print(네이버리뷰)

In [68]:
result=pd.DataFrame(list(zip(네이버평점,네이버감상평)),columns=["평점","감상평"])

In [69]:
result.head(10)

,평점,감상평
0,10,마블 3000만큼 사랑합니다
1,10,Avengers Assemble!
2,10,I'm Ironman.
3,10,관람객아이언맨으로 시작해서 아이언맨으로 끝난다..
4,10,아스가디언즈 오브 갤럭시 벌써부터 기대됨 ㅋㅋㅋㅋ
5,10,히어로물보고 울줄 누가알겠습니까
6,10,인피니티워가 10년을 무너뜨리는 영화라면엔드게임은 10년을 돌아보는 영화입니다
7,10,쥐가다살렸음 ㅋㅋㅋㅋ
8,10,관람객마지막 한타에서 쌌습니다
9,10,10점 안주는 사람들은 마블에 6에서900까지밖에 애정이 없는거 ...
